In [2]:
%%bash
pip install gensim

  Running setup.py bdist_wheel for smart-open: started
  Running setup.py bdist_wheel for smart-open: finished with status 'done'
  Stored in directory: /home/ubuntu/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file: started
  Running setup.py bdist_wheel for bz2file: finished with status 'done'
  Stored in directory: /home/ubuntu/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


mkl-random 1.0.1 requires cython, which is not installed.
mkl-fft 1.0.0 requires cython, which is not installed.


In [329]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(1)

import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from nltk.corpus import wordnet

In [275]:
import gensim
gensim_model = gensim.models.KeyedVectors.load_word2vec_format('./data.bin', binary=True)

In [276]:
weights = gensim_model.wv.syn0
weights.shape

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':


(3000000, 300)

In [277]:
vocab_size = weights.shape[0]
embedding_dim = weights.shape[1]

In [278]:
weights = np.append(weights,np.zeros((1,embedding_dim)),axis=0)
# 末尾にunknown_wordを追加

In [279]:
vocab_size = weights.shape[0]

In [352]:
# wordのindexを取得
print(gensim_model.wv.vocab["'d"].index)
# 100番目のwordを取得
print(gensim_model.wv.index2word[100])

609
company


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [357]:
import re
import nltk
from nltk import word_tokenize

In [282]:
def prepare_sequence(seq):
    vocab = gensim_model.wv.vocab
    idxs = [vocab[w].index if w in vocab else vocab_size - 1 for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [358]:
def sentence2vec(sentence,debug=False):
    w_list = word_tokenize(sentence)
    w_list = [wordnet.morphy(w) if wordnet.morphy(w) is not None else w for w in w_list]
    if debug:
        print(w_list)
    res_seq = prepare_sequence(w_list)
    return res_seq

In [359]:
s = "I'm always fucking dogs."
sentence2vec(s,debug=True)

['I', "'m", 'always', 'fucking', 'dog', '.']


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


tensor([ 2.0000e+01,  2.3600e+02,  3.8800e+02,  2.7891e+04,  2.0430e+03,
         3.0000e+06])

In [360]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.out = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        h = torch.zeros(1, 1, self.hidden_dim)
        c = torch.zeros(1, 1, self.hidden_dim)
#         if cuda:
#             h = h.cuda()
#             c = c.cuda()
        return (h,c)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
#         print(embeds.size())
        
        lstm_output, self.hidden = self.lstm(
            embeds.view(len(sentence),1,-1), self.hidden)
        
        output = self.out(lstm_output.view(len(sentence),-1))
        output = F.tanh(output)
        return output

In [361]:
hidden_dim = 6
out_size = 3
model = LSTMTagger(embedding_dim, hidden_dim, vocab_size, out_size)

In [362]:
# 学習済みパラメータ
model.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(weights).float())

In [363]:
loss_function = nn.MSELoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)

In [289]:
sample = [
    "I'm always fucking you hey",
    "oh my godness"
] 

# forwardのsample
with torch.no_grad():
    inputs = sentence2vec(sample[0])
    output = model(inputs)
    print(output[-1,:])

tensor([ 0.1828, -0.0878,  0.2294])


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [364]:
# dataのload
data_pre = pd.read_csv('./data_preprocessed.csv')

# 2word以上のsentence
data_pre = data_pre[data_pre['words']>=2]

X_orig = data_pre['reg'].as_matrix()
Y_v = data_pre['Valence'].as_matrix()
Y_a = data_pre['Arousal'].as_matrix()
Y_d = data_pre['Dominance'].as_matrix()
Y_orig = np.c_[Y_v,Y_a,Y_d]
a=1
b=5
Y_orig = (2*(Y_orig-a)/(b-a))-1
# [-1,1]で正規化

In [365]:
X = X_orig
Y = Y_orig

In [366]:
train_size = 0.7
dev_size = 0.2
X_train, X_rest, Y_train, Y_rest = train_test_split(X, Y, test_size=1-train_size)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_rest,Y_rest,test_size=1-(train_size+dev_size))

In [ ]:
epochs = 30

loss_ary = []

for epoch in range(epochs):  # again, normally you would NOT do 300 epochs, it is toy data
    loss_sum = 0
    X_train,Y_train = shuffle(X_train,Y_train)
    for sentence, target in zip(X_train,Y_train):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = sentence2vec(sentence)
        
        # Step 3. Run our forward pass.
        y = model(sentence_in)[-1,:]

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(y, torch.tensor(target, dtype=torch.float))
        loss_sum += loss.data[0].item()
        loss.backward()
        optimizer.step()
    print("epoch {0}: loss {1}".format(epoch,loss_sum/len(X_train)))
    loss_ary.append(loss_sum/len(X_train))

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0: loss 0.03251407551398661
epoch 1: loss 0.03148706812360908


In [368]:
X_train

array(['to compare his pieces to those of others is to compare mannequins to living models ',
       '“nathan ” said nepthys  “i think that you just did something very stupid ”',
       'today our unique network of 0 huts and lodges—each a day’s hike apart—stretches 0 miles through the white mountain national forest ',
       ...,
       'wholesalers compete against each other    perhaps you forgot ',
       "more than a toy  it's an equalizer that allows poor kids to ride with rich ",
       'there are also demonstrations of traditional chinese crafts  including silk-weaving  pottery  and p apermaking '],
      dtype=object)